# IV. Convertir le modèle au format GGUF

Au préalable, enregistrer le modèle réalisé lors de l'étape III. dans Kaggle puis intégrer le modèle à ce notebook en cliquant sur "+ add input"

In [7]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

hf_token = user_secrets.get_secret("HF ")
login(token = hf_token)


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [8]:
# Clone d'un code qui convertit des fichiers au format GGUF
%cd /kaggle/working
!git clone --depth=1 https://github.com/ggerganov/llama.cpp.git
%cd /kaggle/working/llama.cpp
!sed -i 's|MK_LDFLAGS   += -lcuda|MK_LDFLAGS   += -L/usr/local/nvidia/lib64 -lcuda|' Makefile
!LLAMA_CUDA=1 conda run -n base make -j > /dev/null

/kaggle/working
fatal: destination path 'llama.cpp' already exists and is not an empty directory.
/kaggle/working/llama.cpp


In [9]:
!python convert-hf-to-gguf.py /kaggle/input/doctor-merged/transformers/version1/1 \
    --outfile /kaggle/working/llama-3-8b-chat-doctor-merged.gguf \
    --outtype f16

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Writing: 100%|██████████████████████████| 16.1G/16.1G [03:14<00:00, 82.4Mbyte/s]


In [13]:
!ls /kaggle/working

llama-3-8b-chat-doctor-merged.gguf  llama.cpp


In [ ]:
# Pousser le modèle sur Hugging Face 

!pip install huggingface_hub

from huggingface_hub import HfApi, HfFolder
from huggingface_hub import create_repo, upload_file

repo_id = "PierreMaxime/llama-3-8b-chat-doctor-merged.gguf"

api = HfApi()

api.create_repo(repo_id=repo_id, private=False)  # Utilisez private=True si vous voulez un dépôt privé


In [20]:
# Chemin vers le fichier modèle à uploader
file_path = "/kaggle/working/llama-3-8b-chat-doctor-merged.gguf"

# Pousser le fichier modèle sur Hugging Face
upload_file(
    path_or_fileobj=file_path,
    path_in_repo="llama-3-8b-chat-doctor-merged.gguf",  # Nom du fichier dans le dépôt
    repo_id=repo_id,
    commit_message="Ajout du modèle fusionné",
    token=hf_token
)


llama-3-8b-chat-doctor-merged.gguf:   0%|          | 0.00/16.1G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/PierreMaxime/llama-3-8b-chat-doctor-merged/commit/ab11ee8fcb448356893f407b0f7421c594abca65', commit_message='Ajout du modèle fusionné', commit_description='', oid='ab11ee8fcb448356893f407b0f7421c594abca65', pr_url=None, pr_revision=None, pr_num=None)